# Advent of Code - Day 5: Binary Boarding

## Load Input data

:::{note}
The input data can be found [here](https://adventofcode.com/2020/day/5).
:::

In [1]:
with open('../../data/advent-of-code/2020/day-5-input') as fid:
    data = fid.readlines()

print(len(data))
print(data[0])

846
FBBFFBBLLL



## Part One

You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of [zones or groups](https://www.youtube.com/watch?v=oAHbLRjF0vo), this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

Start by considering the whole range, rows 0 through 127.
- F means to take the lower half, keeping rows 0 through 63.
- B means to take the upper half, keeping rows 32 through 63.
- F means to take the lower half, keeping rows 32 through 47.
- B means to take the upper half, keeping rows 40 through 47.
- B keeps rows 44 through 47.
- F keeps rows 44 through 45.

The final F keeps the lower of the two, row 44.
The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

- Start by considering the whole range, columns 0 through 7.
- R means to take the upper half, keeping columns 4 through 7.
- L means to take the lower half, keeping columns 4 through 5.
- The final R keeps the upper of the two, column 5.
So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

- BFFFBBFRRR: row 70, column 7, seat ID 567.
- FFFBBBFRRR: row 14, column 7, seat ID 119.
- BBFFBBFRLL: row 102, column 4, seat ID 820.

As a sanity check, look through your list of boarding passes. **What is the highest seat ID on a boarding pass?**



### Solution

In [2]:
import math


def find_seat_info(seat_number, nrows, ncols, row_identifiers, col_identifiers):
    """Function to decode the seat_id when given the seat number"""

    def decode_seat(seat, n_entries, identifiers):
        lo, hi = 0, n_entries
        for letter in seat:
            if letter == identifiers[0]:
                lo, hi = lo, math.floor((lo + hi) / 2)
            elif letter == identifiers[1]:
                lo, hi = math.ceil((lo + hi) / 2), hi

        return lo

    row = decode_seat(seat_number[:7], nrows, row_identifiers)
    col = decode_seat(seat_number[7:], ncols, col_identifiers)
    return {'seat_number': seat_number, 'row': row, 'column': col, 'seat_id': row * 8 + col}

In [3]:
nrows, ncols = 128, 8
row_identifiers = ('F', 'B')
col_identifiers = ('L', 'R')

In [4]:
# Test our function on sample input seat number
find_seat_info('FBFBBFFRLR', nrows, ncols, row_identifiers, col_identifiers)

{'seat_number': 'FBFBBFFRLR', 'row': 44, 'column': 5, 'seat_id': 357}

Everything appears to be working.... Let's loop over all the seat numbers and collect the ids:

In [5]:
ids = [
    find_seat_info(seat, nrows, ncols, row_identifiers, col_identifiers)['seat_id'] for seat in data
]
min(ids), max(ids)

(12, 858)

## Part Two


Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

**What is the ID of your seat?**

### Solution


In [6]:
# Find missing ids
missing_ids = [seat_id for seat_id in range(5, nrows * 8 + 5) if seat_id not in set(ids)]
len(missing_ids)

178

In [7]:
# Loop over missing ids, and find possible candidates
possible_candidates = []
for idx in range(1, len(missing_ids) - 1):
    if not (
        (missing_ids[idx - 1] == missing_ids[idx] - 1)
        and (missing_ids[idx + 1] == missing_ids[idx] + 1)
    ):
        possible_candidates.append(missing_ids[idx])
possible_candidates

[11, 557, 859]

We can use a process of elimination to get the ID of our seat by checking for the ids of the front and back seats:

In [8]:
min(ids), max(ids)

(12, 858)

Since it was pointed out that our seat wasn't at the very front or back, we can eliminate `11` and `859`. So, the id our seat is **`557`**